## 导入必要的库

In [1]:
import torch

## 创建连续的 Tensor (Row-Major)

In [2]:
A = torch.tensor([[1, 2, 3], [4, 5, 6]])

print(f"形状 (Shape): {A.shape}")  # (2, 3)
print(f"步长 (Stride): {A.stride()}")  # (3, 1)

形状 (Shape): torch.Size([2, 3])
步长 (Stride): (3, 1)


### Stride 解释

- 第0维(行)走一步，内存跳3个元素
- 第1维(列)走一步，内存跳1个元素

## 验证内存寻址公式

目标：访问 A[1, 2] (即元素 6)

公式：`Offset + 1 * stride[0] + 2 * stride[1]`

In [3]:
offset = A.storage_offset()
print(f"\nStorage 偏移量 (Offset): {offset}")  # 通常为0
target_idx = offset + 1 * A.stride(0) + 2 * A.stride(1)
print(f"\n元素 A[1, 2] 的值: {A[1, 2]}")
print(f"通过 Stride 计算出的扁平存储索引: {target_idx}")
print(f"Storage 中的对应值: {A.view(-1)[target_idx]}")


Storage 偏移量 (Offset): 0

元素 A[1, 2] 的值: 6
通过 Stride 计算出的扁平存储索引: 5
Storage 中的对应值: 6


## 转置操作 (Transpose) - 零拷贝魔法

In [4]:
B = A.t()  # 转置

print("\n>>> 执行转置 B = A.t() 之后:")
print(f"B 的形状: {B.shape}")  # (3, 2)
print(f"B 的步长: {B.stride()}")  # (1, 3) -> 注意！步长交换了
print(f"A 和 B 内存地址相同吗? {A.data_ptr() == B.data_ptr()}")  # True


>>> 执行转置 B = A.t() 之后:
B 的形状: torch.Size([3, 2])
B 的步长: (1, 3)
A 和 B 内存地址相同吗? True


## 连续性 (Contiguity) 检查

转置后的张量通常不是连续的

In [5]:
print(f"\n原始 A 是否连续: {A.is_contiguous()}")  # True
print(f"转置 B 是否连续: {B.is_contiguous()}")  # False


原始 A 是否连续: True
转置 B 是否连续: False


## 强制连续 (Contiguous) - 发生物理拷贝

In [6]:
C = B.contiguous()
print("\n>>> 执行 C = B.contiguous() 之后:")
print(f"C 的步长: {C.stride()}")  # (2, 1) -> 恢复默认的行优先布局
print(f"B 和 C 内存地址相同吗? {B.data_ptr() == C.data_ptr()}")  # False (发生了拷贝)


>>> 执行 C = B.contiguous() 之后:
C 的步长: (2, 1)
B 和 C 内存地址相同吗? False


## 总结

### Stride 的作用

- Stride 定义了在多维张量中移动时需要跳过的内存元素数量
- 通过改变 stride，可以实现零拷贝的视图操作（如转置）

### Contiguity 的重要性

- 连续的张量在内存中按行优先顺序存储
- 某些操作（如 `view`）要求张量是连续的
- 使用 `contiguous()` 可以创建连续的副本，但会触发内存拷贝

### 性能考虑

- 零拷贝操作（改变 stride）速度快，不占用额外内存
- 但非连续的内存访问模式可能影响某些操作的性能
- 需要在内存使用和访问效率之间权衡